In [2]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

def writeLog(row):
    with open('log.txt', 'a') as outfile:
        outfile.write(row + '\n')

def getErrMsg(e):
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    errMsg = "[{}] {}".format(error_class, detail)
    return errMsg

# IMDb social feature
* For each movie, we collect the ratings given by the users of IMDb and compute the average rating. Also, we collect the votes obtained on IMDb. Next, we characterize it by concatenating the average rating and the vote together to form a 2-dimensional vector. Similarly, we normalize the values of each dimension into [0,1]. Next, we combine the vectors of social features obtained from both Instagram and IMDb together to derive a feature vector of the movie. Finally, we input the combined vector into a neural network with one embedding layer to derive the social feature vector.

In [3]:
df = pd.read_csv('./ratings.csv')
df

,Account,tconst,averageRating,numVotes
0,21bridgesmovie,tt8688634,6.6,18694
1,47metersdown,tt7329656,5.0,12683
2,abeautifuldaymovie,tt3224458,7.4,29979
3,abominablemovie,tt6324278,7.0,18487
4,adastramovie,tt2935510,6.6,147914
...,...,...,...,...
160,wrinklestheclown,tt9097270,4.7,537
161,xmenmovies,tt6565702,5.8,125765
162,yardiefilm,tt5862902,5.8,2248
163,yesterdaymovie,tt8079248,6.9,79700


# Instagram social feature
* To characterize the activeness of a movie on Instagram, for each movie account, we extract the number of followers and the number of posts. The number of follows may reflect how popular the movie is while the number of posts may reflect how often the move company promotes its movie. Also, to measure movie’s recognition, for each movie, we collect the number of likes and the number of comments obtained on Instagram. Thus, we obtain a 4-dimensional vector, where the values of each dimension are normalized into [0,1]. 

In [4]:
import instaloader

In [5]:
accs = df['Account'].tolist()
print(len(accs), accs[:10])
# uids = []
# mediacount = []
# followerNum = []
# likecount = []
# commentcount = []

165 ['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']


In [6]:
# instaloader instance and login
L = instaloader.Instaloader()
L.login('yuwei8810', 'r07725005') # (login)

In [49]:
uids.append(9188405541)
mediacount.append(58)
followerNum.append(17916)

In [50]:
POST_PATH = '../Posts_165/triplefrontier/'
ls = os.listdir(POST_PATH)
print(len(ls))

likes = 0
comments = 0

for folder in ls:
    post = read_json(POST_PATH + folder + '/post.json')
    likes += post['like_count']
    comments += post['comment_count']
    
likecount.append(likes)
commentcount.append(comments)

58


In [54]:
idx = 153

for acc in accs[idx:]:
    
    print(idx)
    profile = instaloader.Profile.from_username(L.context, acc)
    print(profile.userid, profile.username)
    print(profile.mediacount, profile.followers)
    
    likes = 0
    comments = 0
    posts = profile.get_posts()
    for post in posts:
        likes += post.likes
        comments += post.comments
        
    uids.append(profile.userid)
    mediacount.append(profile.mediacount)
    followerNum.append(profile.followers)
    likecount.append(likes)
    commentcount.append(comments)
    
    print(likes, comments)
    print('==============================')
    idx += 1

153
1430177837 uglydolls
1165 47549
617499 15798
154
5498376142 uncutgems
76 42834
367712 6715
155
5956895944 underthesilverlake
22 1896
7615 212
156
7701828816 usmovie
156 63877
752473 21792
157
18513735865 wavesmovie
68 11650
100504 1762
158
6438477602 whatmenwant
114 22682
99022 3125
159
7472215985 wonderparkmovie
92 7033
81359 1548
160
16149244607 wrinklestheclown
72 19564
108787 3289
161
401420930 xmenmovies
1333 1036371
12189097 115918
162
6552487116 yardiefilm
47 6574
25558 1436
163
10390112691 yesterdaymovie
118 45500
500137 8931
164
9139732106 zombieland
151 97918
1013914 11326


In [55]:
print(len(uids), len(mediacount), len(followerNum), len(likecount), len(commentcount))

165 165 165 165 165


In [53]:
accs[153]

'uglydolls'

In [46]:
write_json([uids, mediacount, followerNum, likecount, commentcount], './temp.json')

In [47]:
print(uids[-1], mediacount[-1], followerNum[-1], likecount[-1], commentcount[-1])

8197720659 148 510693 4990299 59024


In [32]:
i = 64
uids = uids[:i]
mediacount = mediacount[:i]
followerNum = followerNum[:i]
likecount = likecount[:i]
commentcount = commentcount[:i]

In [56]:
df['Uid'] = uids
df['Posts'] = mediacount
df['Followers'] = followerNum
df['Likes'] = likecount
df['Comments'] = commentcount
df

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments
0,21bridgesmovie,tt8688634,6.6,18694,8600709558,81,15595,73032,1468
1,47metersdown,tt7329656,5.0,12683,4773990401,61,38297,234154,5986
2,abeautifuldaymovie,tt3224458,7.4,29979,9457986739,133,32213,245879,3323
3,abominablemovie,tt6324278,7.0,18487,8751027688,134,19847,152470,2489
4,adastramovie,tt2935510,6.6,147914,8014594288,118,19262,349380,8189
...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,tt9097270,4.7,537,16149244607,72,19564,108787,3289
161,xmenmovies,tt6565702,5.8,125765,401420930,1333,1036371,12189097,115918
162,yardiefilm,tt5862902,5.8,2248,6552487116,47,6574,25558,1436
163,yesterdaymovie,tt8079248,6.9,79700,10390112691,118,45500,500137,8931


In [74]:
df.to_csv('./IGdata.csv', index = 0)

## Wrong accounts

In [59]:
pd.set_option('display.max_rows', 500)

In [58]:
acc = 'thefastsaga'

profile = instaloader.Profile.from_username(L.context, acc)
print(profile.userid, profile.username)
print(profile.mediacount, profile.followers)

likes = 0
comments = 0
posts = profile.get_posts()
for post in posts:
    likes += post.likes
    comments += post.comments

print(likes, comments)

211430315 thefastsaga
876 3933998
31936779 355189


In [73]:
df

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments
0,21bridgesmovie,tt8688634,6.6,18694,8600709558,81,15595,73032,1468
1,47metersdown,tt7329656,5.0,12683,4773990401,61,38297,234154,5986
2,abeautifuldaymovie,tt3224458,7.4,29979,9457986739,133,32213,245879,3323
3,abominablemovie,tt6324278,7.0,18487,8751027688,134,19847,152470,2489
4,adastramovie,tt2935510,6.6,147914,8014594288,118,19262,349380,8189
5,adogsjourneymovie,tt8385474,7.4,12969,2226168062,212,59678,528166,11321
6,aftermathmovie,tt5977276,6.3,13315,4736097525,51,2511,17453,380
7,aftermovie,tt4126476,5.4,24503,7165204352,55,2288226,24096968,459256
8,ahiddenlifefilm,tt5827916,7.6,6785,14231496569,37,2274,10933,187
9,alitamovie,tt0437086,7.3,193030,4860003337,353,130827,1867811,26006


In [72]:
df.loc[df['Account'] == 'fastandfuriousmovie']

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments
52,fastandfuriousmovie,tt6806448,6.5,134511,211430315,876,3933998,31936779,355189


In [71]:
df.loc[52,'Uid'] = profile.userid
df.loc[52,'Posts'] = profile.mediacount
df.loc[52,'Followers'] = profile.followers
df.loc[52,'Likes'] = likes
df.loc[52,'Comments'] = comments

# Normalize

In [3]:
df = pd.read_csv('./IGdata.csv')
df

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments
0,21bridgesmovie,tt8688634,6.6,18694,8600709558,81,15595,73032,1468
1,47metersdown,tt7329656,5.0,12683,4773990401,61,38297,234154,5986
2,abeautifuldaymovie,tt3224458,7.4,29979,9457986739,133,32213,245879,3323
3,abominablemovie,tt6324278,7.0,18487,8751027688,134,19847,152470,2489
4,adastramovie,tt2935510,6.6,147914,8014594288,118,19262,349380,8189
...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,tt9097270,4.7,537,16149244607,72,19564,108787,3289
161,xmenmovies,tt6565702,5.8,125765,401420930,1333,1036371,12189097,115918
162,yardiefilm,tt5862902,5.8,2248,6552487116,47,6574,25558,1436
163,yesterdaymovie,tt8079248,6.9,79700,10390112691,118,45500,500137,8931


In [4]:
df['averageRating_norm'] = df['averageRating']/10
df

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments,averageRating_norm
0,21bridgesmovie,tt8688634,6.6,18694,8600709558,81,15595,73032,1468,0.66
1,47metersdown,tt7329656,5.0,12683,4773990401,61,38297,234154,5986,0.50
2,abeautifuldaymovie,tt3224458,7.4,29979,9457986739,133,32213,245879,3323,0.74
3,abominablemovie,tt6324278,7.0,18487,8751027688,134,19847,152470,2489,0.70
4,adastramovie,tt2935510,6.6,147914,8014594288,118,19262,349380,8189,0.66
...,...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,tt9097270,4.7,537,16149244607,72,19564,108787,3289,0.47
161,xmenmovies,tt6565702,5.8,125765,401420930,1333,1036371,12189097,115918,0.58
162,yardiefilm,tt5862902,5.8,2248,6552487116,47,6574,25558,1436,0.58
163,yesterdaymovie,tt8079248,6.9,79700,10390112691,118,45500,500137,8931,0.69


In [5]:
cols = ['numVotes', 'Posts', 'Followers', 'Likes', 'Comments']
cols

['numVotes', 'Posts', 'Followers', 'Likes', 'Comments']

In [6]:
from sklearn.preprocessing import MinMaxScaler
def normalizing(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    print('Data max:', scaler.data_max_)
    print('Data min:', scaler.data_min_)
    print('Data range:', scaler.data_range_)
    return scaler.transform(data)

In [7]:
for col in cols:
    arr = np.asarray(df[col].tolist()).reshape(-1, 1)
    df[col+'_norm'] = normalizing(arr)

Data max: [701017.]
Data min: [399.]
Data range: [700618.]
Data max: [3763.]
Data min: [20.]
Data range: [3743.]
Data max: [12910480.]
Data min: [547.]
Data range: [12909933.]
Data max: [3.67168106e+08]
Data min: [3673.]
Data range: [3.67164433e+08]
Data max: [2941753.]
Data min: [124.]
Data range: [2941629.]


In [8]:
df

,Account,tconst,averageRating,numVotes,Uid,Posts,Followers,Likes,Comments,averageRating_norm,numVotes_norm,Posts_norm,Followers_norm,Likes_norm,Comments_norm
0,21bridgesmovie,tt8688634,6.6,18694,8600709558,81,15595,73032,1468,0.66,0.026113,0.016297,0.001166,0.000189,0.000457
1,47metersdown,tt7329656,5.0,12683,4773990401,61,38297,234154,5986,0.50,0.017533,0.010954,0.002924,0.000628,0.001993
2,abeautifuldaymovie,tt3224458,7.4,29979,9457986739,133,32213,245879,3323,0.74,0.042220,0.030190,0.002453,0.000660,0.001087
3,abominablemovie,tt6324278,7.0,18487,8751027688,134,19847,152470,2489,0.70,0.025817,0.030457,0.001495,0.000405,0.000804
4,adastramovie,tt2935510,6.6,147914,8014594288,118,19262,349380,8189,0.66,0.210550,0.026182,0.001450,0.000942,0.002742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,tt9097270,4.7,537,16149244607,72,19564,108787,3289,0.47,0.000197,0.013893,0.001473,0.000286,0.001076
161,xmenmovies,tt6565702,5.8,125765,401420930,1333,1036371,12189097,115918,0.58,0.178936,0.350788,0.080235,0.033188,0.039364
162,yardiefilm,tt5862902,5.8,2248,6552487116,47,6574,25558,1436,0.58,0.002639,0.007213,0.000467,0.000060,0.000446
163,yesterdaymovie,tt8079248,6.9,79700,10390112691,118,45500,500137,8931,0.69,0.113187,0.026182,0.003482,0.001352,0.002994


In [9]:
df.to_csv('./social_norm.csv', index = 0)